<a href="https://colab.research.google.com/github/wdpressplus-bigdata/wdpressplus-bigdata/blob/main/notebooks/7-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# p.273 List 7.2
import pathlib
import requests

def download_file(filename):
    prefix = 'https://github.com/wdpressplus-bigdata/uscrn/raw/main'
    # prefix = 'https://www1.ncdc.noaa.gov/pub/data/uscrn/products/subhourly01'
    r = requests.get(f"{prefix}/2020/{filename}")
    r.raise_for_status()
    path = pathlib.Path('./raw')
    path.mkdir(parents=True, exist_ok=True)
    with open(path / filename, 'wb') as f:
        f.write(r.content)
    print(f"Saved {path / filename}")

FILES = [
    'CRNS0101-05-2020-AK_Aleknagik_1_NNE.txt',
    'CRNS0101-05-2020-AK_Bethel_87_WNW.txt',
]
for filename in FILES:
    download_file(filename)

Saved raw/CRNS0101-05-2020-AK_Aleknagik_1_NNE.txt
Saved raw/CRNS0101-05-2020-AK_Bethel_87_WNW.txt


In [2]:
# p.274
!pip install pandas    

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 4.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 4.1 MB/s eta 0:00:0000:0100:01


In [3]:
import glob
import pandas as pd

def read_tables():
    for path in glob.glob('./raw/*.txt'):
        yield pd.read_table(path, delimiter='\s+', header=None, dtype='str')

df = pd.concat(read_tables())
df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,26656,20200101,0005,20191231,1505,3,-164.08,61.35,-20.8,0.0,...,C,0,80,0,-99.000,-9999.0,1005,0,6.69,0
1,26656,20200101,0010,20191231,1510,3,-164.08,61.35,-20.8,0.0,...,C,0,79,0,-99.000,-9999.0,1005,0,7.64,0


In [4]:
# p.275
df1 = df[[0, 1, 2, 8]]
df1.columns = ['wbanno', 'utc_date', 'utc_time', 'temperature']
df1.head(2)

,wbanno,utc_date,utc_time,temperature
0,26656,20200101,0005,-20.8
1,26656,20200101,0010,-20.8


In [5]:
df2 = df1.copy()
df2.index = pd.to_datetime(df2['utc_date'] + df2['utc_time'])
df2.drop(columns=['utc_date', 'utc_time'], inplace=True)
df2.head(2)

,wbanno,temperature
2020-01-01 00:05:00,26656,-20.8
2020-01-01 00:10:00,26656,-20.8


In [6]:
# p.276
df2.describe().T

,count,unique,top,freq
wbanno,192360,2,26656,96180
temperature,192360,569,0.3,1182


In [7]:
df2['temperature'] = df2['temperature'].astype('float')
df2.describe().T

,count,mean,std,min,25%,50%,75%,max
temperature,192360.0,-9.160542,325.703007,-9999.0,-5.0,3.7,10.2,24.8


In [8]:
# p.277
df3 = df2.copy()
df3.loc[df3['temperature'] == -9999.0, 'temperature'] = None
df3.describe().T

,count,mean,std,min,25%,50%,75%,max
temperature,192156.0,1.445045,11.572591,-32.0,-5.0,3.7,10.2,24.8


In [9]:
# p.278
!pip install pyspark==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 1.3 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 3.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=4283bb7dd4271edeb3ec72b758ed7041bca93cebc8a85c7b1f23ef25bf5f9915
  Stored in directory: /home/ubuntu/.cache/pip/wheels/ea/21/84/970b03913d0d6a96ef51c34c878add0de9e4ecbb7c764ea21f
Successfully built pyspark


In [ ]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

In [ ]:
# p.279
rdd = spark.sparkContext.textFile('./raw/*')
rdd.take(2)

In [ ]:
rdd.count()

In [ ]:
from datetime import datetime, timezone
from pyspark.sql import Row

def parse_line(line):
    f = line.split()
    wbanno = f[0]
    dt = datetime.strptime(f[1] + f[2], '%Y%m%d%H%M')
    dt = dt.replace(tzinfo=timezone.utc)
    temperature = None if f[8] == '-9999.0' else float(f[8])
    return Row(timestamp=dt, wbanno=wbanno, temperature=temperature)

rows = rdd.map(parse_line)
rows.take(2)

In [ ]:
# p.280
df = rdd.map(parse_line).toDF()
df

In [ ]:
spark.conf.set("spark.sql.session.timeZone", 'UTC')

In [ ]:
df.show(2)

In [ ]:
# p.281
df.describe().show()

In [ ]:
df.createOrReplaceTempView('uscrn')

In [ ]:
query = '''
SELECT
  wbanno,
  min_by(timestamp, temperature) timestamp_min,
  min(temperature) t_min,
  max_by(timestamp, temperature) timestamp_max,
  max(temperature) t_max
FROM
  uscrn
GROUP by
  1
'''
spark.sql(query).show()

In [ ]:
# p.283
df.write.save('./uscrn-parquet')

In [ ]:
!ls ./uscrn-parquet

In [ ]:
df = spark.read.load('./uscrn-parquet')
df.groupBy('wbanno').avg('temperature').show()

In [ ]:
# p.284
df = spark.read.load('./uscrn-parquet')
df1 = df.where("timestamp >= '2020-01-01' AND timestamp < '2020-04-01'")
df1.count()

In [ ]:
df1.coalesce(1).write.save('./export', format='csv', header=True)

In [ ]:
!ls ./export

In [ ]:
!cp ./export/*.csv ~/Home/Desktop/

In [ ]:
# p.285 Column
!pip install pyarrow

In [ ]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.session.timeZone", 'UTC')

df = spark.read.load('./uscrn-parquet')
df1 = df.groupBy('timestamp').avg().toPandas()
df1.sort_values(by='avg(temperature)', ascending=False).head(2)

In [ ]:
import pandas as pd
df = pd.read_parquet('./uscrn-parquet')

df1 = df.groupby('timestamp').mean()
df1.sort_values(by='temperature', ascending=False).head(2)